In [ ]:
import pandas as pd

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# Add project root to sys.path for module imports
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from src.utils.data_processing import create_train_and_test_datasets
from src.utils.models import MLP
from src.utils.train_functions import train, evaluate

# Load data

## Load parameter data

In [ ]:
building_parameters = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/building_parameters/building_parameters.csv")
building_parameters.set_index("building_id", inplace=True)

## ResStock data

In [ ]:
processed_resstock = pd.read_csv("/Users/cocoloco/Library/Mobile Documents/com~apple~CloudDocs/Documents/ICAI/4o/AI Lab/data/resstock/resstock_processed.csv")
processed_resstock.set_index("bldg_id", inplace=True)

# Create datasets

In [ ]:
train_dataset, test_dataset = create_train_and_test_datasets(
    building_parameters=building_parameters,
    processed_resstock=processed_resstock,
    test_size=0.2,
)

In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define model

In [ ]:
# HYPERPARAMETERS
# model hyperparameters
input_size = len(processed_resstock.columns)
hidden_sizes = [64, 32]
output_size = len(building_parameters.columns)

# training hyperparameters
num_epochs = 100
learning_rate = 0.001

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
parameter_prediction_model = MLP(
    input_size=input_size,
    hidden_sizes=hidden_sizes,
    output_size=output_size,
).to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(parameter_prediction_model.parameters(), lr=learning_rate)

# Train model

In [ ]:
model_name = "parameter_prediction_model_1.pth"
writer = SummaryWriter(log_dir=f"runs/{model_name.replace('.pth', '')}")

In [ ]:
train(
    model=parameter_prediction_model,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    writer=writer,
    verbose=True
)

# Evaluate model

In [ ]:
predictions, eval_loss = evaluate(
    model=parameter_prediction_model,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
)